# HW4 布尔查询之BSBI与索引压缩

由于BSBI与索引压缩对于索引的存储方式有着一定的要求，所以之前的框架不再适用，本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

# 实验报告

## 代码实现
注：部分代码块的输出中有些数字，这些数字是输出的运行时间。

&nbsp;

1.**完善了IdMap类**，实现了str和int的转换，实现了当要检索的str不存在时，添加该内容。

&nbsp;

2.**实现了BSBIIndex的解析部分parse_block()并对其进行了测试。**遍历文件夹下的所有文件，建立包含所有文件的[term, doc]的列表td_pairs。测试了文件同名但处于不同块中时的情况，此时的文件编号应该是不同的。

&nbsp;

3.**实现了对.index文件的写操作。**

（1）创建InvertedIndex的子类InvertedIndexWriter，使用这个子类的append()成员方法，实现对.index文件的添加操作，该方法主要讲term的倒排索引列表编码并写入.index文件中。

（2）在BSBIIndex类中添加invert_write()成员方法，该方法遍历td_pairs中的所有内容，为所有term建立倒排索引列表，最后使用index.append()将结果写入.index文件中。

&nbsp;

4.**实现了各个块的合并操作。**

（1）完善了InvertedIndexIterator类。

在\_\_enter\_\_方法下，创建了对terms（倒排索引表中的词项）的迭代器，并让index_file的文件指针指向文件头。在\_\_next\_\_方法下，先获得下一个term，再利用metadata，从index_file中读出相应的倒排索引列表的编码。

（2）利用heapq实现了BSBIIndex中的merge()方法。

该方法利用最小堆，先将所有迭代器的第一个\[ (term, postings_dict), i \] （i是迭代器的下标）压入堆中，从而完成对最小堆的初始化。之后当堆不为空时，一直循环读堆顶的数据并压入相应的迭代器的下一对数据，当读到的堆顶的term与前一个不同时，将前面的postings_list做合并，并写入merge_index中，然后开始对下一个term做操作。

&nbsp;

5.**实现了文档的索引、查询操作。**

（1）完善了InvertedIndexMapper类。

使用term的metadata，将文件指针指到相应的位置，并读取对应的长度，将编码解码后返回。

（2）实现了两个列表的交集操作。

（3）实现了BSBIIndex的retrieve()成员函数。

打开一个InvertedIndexMapper类，对query的每一个单词，利用InvertedIndexMapper类获得相应的postings_list，对所有倒排索引列表取交集，并返回文档的名称列表。

&nbsp;

6.**实现了可变长编码**（为了解决0的编码解码问题，本程序将所有docID都+1再进行解码，最后解码的结果也都-1）

（1）编码

使用位运算，对原来的docID二进制，每7位为1组进行编码。将postings_list中所有docID编码后拼接而成的内容转化为字节流后返回。

（2）解码

每8位进行解析，若首位为0则当前docID结束，进行下一个docID的计算。

&nbsp;

7.**实现了Gamma encoding**（为了解决0和1的编码解码问题，本程序将所有docID都+2再进行解码，最后解码的结果也都-2）

（1）编码

按docID一个个编码，然后将编码的结果拼接后变成字节流返回。
对单个docID，获得该docID二进制的长度length，除去首位，将剩余的二进制左移length位，并将低(length-1)位置为1。

注：gamma encoding原本应该是高位是一元编码，低位是原来的二进制除去首位，但为了后面解码方便，本程序将一元编码放在了低位。

（2）解码

先将字节流转换为整数，然后对整数进行解码。

gamma encoding解码的关键在于找到第一位0。为加快解码速度，本程序每8位查找整数中的0，若低8位的值为255，说明这8位都是1，继续找下一个8位，若不为255，说明有0出现，这时再按2位2位找，最后在2位中找到为0的那一位。找到第一个0后，读取整数中相应的位数，加上首位的1，即得到一个docID。接下来就循环上述操作，直到整数为0。


In [4]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
import time
import math

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [ ]:
import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [ ]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [12]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [ ]:
sorted(os.listdir('pa1-data/0'))[:10]

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [ ]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [ ]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [5]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id.keys():  # 没有在字典中，就分别在字典和list中添加
            self.str_to_id[s] = len(self.id_to_str)
            self.id_to_str.append(s)
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [6]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [7]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        # 转化为字节流，更方便得进行读写操作
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [8]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(len(x))
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
12
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [9]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    metadata_file: term，在索引文件中的起始位置，在这个list中包含的文件的个数，在这个list中编码的长度
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    index_file: item按顺序（文件在postings_list中的顺序）排列
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding = None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}  # 倒排索引 三元组
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards
                                # 包含的terms
    
    """enter()和exit()的作用是可以把类当成文件进行IO操作"""
    # 使用with语句时会进行enter()
    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    # 退出with时会进行exit()
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [10]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding  # 编码类型

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        # 保存doc_id_map和term_id_map到相应的输出目录下
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        # 从相应的输出目录下读取doc_id_map和term_id_map
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents  使用parse_block解析文档
        calls invert_write, which inverts each block and writes to a new index  使用invert_write构建倒排索引表
        then saves the id maps and calls merge on the intermediate indices
        """
        time_start=time.time()
        i = 0
        # 遍历所有数据文件
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):  # next() 返回迭代器的下一个项目
            # block_dir_relative: 块的相对路径
            start = time.time()
            td_pairs = self.parse_block(block_dir_relative)  # 对文档进行解析，将文档转换为id对   获得term和doc的映射
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)  # 就是list的append
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:  # index是一个InvertedIndexWriter类的实例
                self.invert_write(td_pairs, index)  # 使用td_pairs写一个倒排索引表,将表写入index中
                td_pairs = None
            end = time.time()
            print("第"+str(i)+"块时间为: ", end-start)  # 输出当前块的时间
            i += 1
        self.save()  # 保留两个映射
        time_end=time.time()
        print('索引构建时间：', time_end-time_start)  # 输出所有快构建的总时间
        time_start=time.time()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:  # merged_index是一个InvertedIndexWriter类的实例
            # 使用contextlib可以同时打开多个文件
            with contextlib.ExitStack() as stack:
                # indices是InvertedIndexIterator类实例的列表
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)  # 很多个小的索引文件合并成大的索引文件
        time_end=time.time()
        print('索引合并时间：', time_end-time_start)  # 输出合并的时间

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [11]:
class BSBIIndex(BSBIIndex):  
    # 文档解析
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        td_pairs = []  # 初始化结果list
        son_data_dir = os.path.join(self.data_dir, block_dir_relative)  # 子目录路径
        # 获得子目录中所有文件的list
        for root, dirs, files in os.walk(son_data_dir):
            # 遍历所有文件
            for text_file in sorted(files):
                text_file_dir = os.path.join(son_data_dir, text_file)
                with open(text_file_dir, 'r') as f:
                    text = f.read().split()
                # 构建td列表
                for word in set(text):
                    td_pairs.append((self.term_id_map[word], self.doc_id_map[os.path.join(block_dir_relative, text_file)]))
        return td_pairs
        ### End your code

观察函数在测试数据上是否正常运行

In [12]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [13]:
toy_dir = 'toy-data/'
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (2, 1),
 (6, 1),
 (7, 1),
 (8, 1)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [14]:
### Begin your code
BSBI_instance.parse_block('2')  # 测试文件同名但目录不同的情况
### End your code

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 3),
 (2, 3),
 (6, 3),
 (7, 3),
 (8, 3)]

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [15]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        # 需要结合with使用
        postings_list = sorted(postings_list)  # postings_list理应是有序的，但为了安全起见，还是再排一遍序
        after_encoding = self.postings_encoding.encode(postings_list)  # 获得编码后的bytes
        start_position_in_index_file = self.index_file.tell()  # 当前文档的长度
        number_of_postings_in_list = len(postings_list)  # 列表的长度
        length_in_bytes_of_postings_list = len(after_encoding)  # bytes的长度
        self.terms.append(term)  # 添加这个term
        # 添加这个term的metadata
        self.postings_dict[term] = (start_position_in_index_file, number_of_postings_in_list, length_in_bytes_of_postings_list)
        self.index_file.write(after_encoding)  # 将编码后的内容写入.index文件
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [16]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    # print(index.terms)
    # print(index.postings_dict)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

# 确定terms和postings_dict是否保存
with InvertedIndex('test', directory='tmp/') as index:
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [17]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        postings_list = {}
        # 为所有term在字典中创建空列表
        for term in [x[0] for x in td_pairs]:
            postings_list[term] = []
        # 获得所有term的倒排索引表
        for term, doc in td_pairs:
            postings_list[term].append(doc)
        # 将倒排表写入.index文件中
        for term in sorted(postings_list.keys()):
            index.append(term, postings_list[term])
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [18]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
block_dir_relative = '0'
td_pairs = BSBI_instance.parse_block(block_dir_relative)
print(td_pairs)
index_id = block_dir_relative
with InvertedIndexWriter(index_id, directory = BSBI_instance.output_dir, 
                         postings_encoding = BSBI_instance.postings_encoding) as index:
    BSBI_instance.invert_write(td_pairs, index)
    index.index_file.seek(0)
    print(index.postings_encoding.decode(index.index_file.read()))
with InvertedIndex(index_id, directory = BSBI_instance.output_dir, 
                   postings_encoding = BSBI_instance.postings_encoding) as index:
    print(index.terms)
    print(index.postings_dict)
    print(index.postings_encoding.decode(index.index_file.read()))
### End your code

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 1), (2, 1), (6, 1), (7, 1), (8, 1)]
[0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 2, 8), 3: (16, 1, 4), 4: (20, 1, 4), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4), 8: (36, 1, 4)}
[0, 0, 0, 1, 0, 0, 1, 1, 1, 1]


### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [19]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.term_iterator = iter(self.terms)  # 创建对倒排索引表中词项的迭代器
        self.index_file.seek(0)  # 令文件指针指向文件头
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):  # 迭代操作
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        """
        (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)"""
        ### Begin your code
        this_term = next(self.term_iterator)  # 获得下一个词项
        # 获得该词项的倒排索引列表的编码长度
        length_in_bytes_of_postings_list = self.postings_dict[this_term][2]
        # 从文件中读取相应长度
        postings_list = self.index_file.read(length_in_bytes_of_postings_list)
        return (this_term, self.postings_encoding.decode(postings_list))
        ### End your code

    def delete_from_disk(self):  # 从磁盘上删除小的文件
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [20]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
block_dir_relative = '0'
td_pairs = BSBI_instance.parse_block(block_dir_relative)
print(td_pairs)
index_id = block_dir_relative
with InvertedIndexWriter(index_id, directory = BSBI_instance.output_dir, 
                         postings_encoding = BSBI_instance.postings_encoding) as index:
    BSBI_instance.invert_write(td_pairs, index)
    index.index_file.seek(0)
    print(UncompressedPostings.decode(index.index_file.read()))
    with InvertedIndexIterator(index_id, directory = BSBI_instance.output_dir, 
                         postings_encoding = BSBI_instance.postings_encoding) as Iterator:
        while(1):
            try:
                print(next(Iterator))
            except:
                break
### End your code

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 1), (2, 1), (6, 1), (7, 1), (8, 1)]
[0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
(0, [0])
(1, [0])
(2, [0, 1])
(3, [0])
(4, [0])
(5, [1])
(6, [1])
(7, [1])
(8, [1])


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [21]:
import heapq  # 实现堆排序的数据结构
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Gray Birch', 50),
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

test_list = [[1,6,7], [2,3,4]]
for item in heapq.merge(*test_list):
    print(item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)
1
2
3
4
6
7


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [22]:
import heapq
from heapq import heappush, heappop
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        block_number = len(indices)  # 块的个数
        heap = []  # 要维护的最小堆
        # 初始化最小堆
        for i in range(block_number):
            heappush(heap, [next(indices[i]), i])
        # 当堆不为空时
        while heap:
            cur_all_postings = []  # 文档列表的list
            cur_postings = heappop(heap)  # 获得堆顶
            # 若对应的迭代器不为空，则压入堆中，否则将对应文件标志为删除
            try:
                heappush(heap, [next(indices[cur_postings[1]]), cur_postings[1]])
            except:
                indices[cur_postings[1]].delete_from_disk()
            # 获得当前词项的编号
            cur_item = cur_postings[0][0]
            cur_all_postings.append(cur_postings[0][1])
            # 使用最小堆，获得所有当前词项的文档列表
            while heap :
                # 堆顶的词项不是当前词项，则退出
                if cur_item != heap[0][0][0]:
                    break
                postings = heappop(heap)
                cur_all_postings.append(postings[0][1])
                try:
                    heappush(heap, [next(indices[postings[1]]), postings[1]])
                except:
                    indices[postings[1]].delete_from_disk()
            # 合并
            merged_index.append(cur_item, [x for x in heapq.merge(*cur_all_postings)])
        ### End your code

首先确保它在测试数据上正常运行

In [23]:
toy_BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', postings_encoding = UncompressedPostings)
toy_BSBI_instance.index()

第0块时间为:  0.0039310455322265625
第1块时间为:  0.0225677490234375
第2块时间为:  0.004025697708129883
索引构建时间： 0.033577919006347656
索引合并时间： 0.03691291809082031


接下来对整个数据集构建索引

In [89]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', postings_encoding = UncompressedPostings)
BSBI_instance.index()

第0块时间为:  22.949820280075073
第1块时间为:  20.4154269695282
第2块时间为:  24.456767320632935
第3块时间为:  21.595614910125732
第4块时间为:  23.422343730926514
第5块时间为:  20.017051458358765
第6块时间为:  18.818127632141113
第7块时间为:  16.056620597839355
第8块时间为:  16.177923440933228
第9块时间为:  15.2453293800354
索引构建时间： 199.5573170185089
索引合并时间： 22.398460865020752


如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [ ]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', postings_encoding = UncompressedPostings)
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [24]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        # 获得当前term的metadata
        metadata = self.postings_dict[term]
        self.index_file.seek(metadata[0])  # 将文件指针定位到对应的位置
        # 读取相应的长度，解码后返回
        return self.postings_encoding.decode(self.index_file.read(metadata[2]))
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [25]:
### Begin your code
query = ['you', 'bye']
with InvertedIndexMapper(index_name = 'BSBI', directory = 'toy_output_dir') as Mapper:
    for word in query:
        print(word, toy_BSBI_instance.term_id_map[word])
        print(Mapper[toy_BSBI_instance.term_id_map[word]])
### End your code

you 2
[0, 1, 2, 4, 5]
bye 9
[2, 3]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [26]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    i = 0
    j = 0
    result = []
    while(i < len(list1) and j < len(list2)):
        if list1[i] > list2[j]:
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            result.append(list1[i])
            i += 1
            j += 1
    return result
    ### End your code

简单的测试样例

In [27]:
### Begin your code
print(sorted_intersect([0,1,7,9,10], [0,2,3,10]))
### End your code

[0, 10]


现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [28]:
# %%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        DocIds = []
        result = []
        with InvertedIndexMapper(index_name = self.index_name, directory = self.output_dir, 
                                 postings_encoding=self.postings_encoding) as Mapper:
            # 遍历query中的所有单词
            for word in set(query.split()):
                if self.term_id_map[word] not in Mapper.terms:
                    DocIds = []
                    break
                this_docId = Mapper[self.term_id_map[word]]
                if DocIds != []:
                    DocIds = sorted_intersect(DocIds, this_docId)
                else:
                    DocIds = this_docId
                if DocIds == []:
                    break
        for doc_Id in DocIds:
            result.append(self.doc_id_map[doc_Id])
        return result
        ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [29]:
toy_BSBI_instance = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir', postings_encoding = UncompressedPostings)
toy_BSBI_instance.retrieve('thank you')

['0\\fine.txt', '2\\fine.txt']

In [30]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', postings_encoding = UncompressedPostings)
BSBI_instance.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [31]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [32]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = sorted([os.path.normpath(path) for path in BSBI_instance.retrieve(query)])
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [252]:
set(my_results) - set(reference_results)

set()

In [253]:
set(reference_results) - set(my_results)

set()

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [65]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [30]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def do_number_encoding(num):
        num += 1  # 为了编码0, 所有数字加1
        length = 1
        div = 2 ** 7
        result = num % div
        num //= div
        # 数字不为0时，一直循环
        while num > 0:
            reminder = num % div  # 7位为1组
            num //= div  # 整数左移7位
            reminder = reminder | div  # 或上 1000 0000
            result = result << 8
            result = reminder | result
            length += 1
        return length, result
    
    @staticmethod
    def do_number_decoding(num):
        div = 2 ** 8
        highest_pos = 2 ** 7
        result = []
        this_number = 0
        while num > 0:
            reminder = num % div
            this_number = this_number << 7
            this_number = this_number | (reminder & (highest_pos - 1))
            # 当前posting的最后一位
            if reminder < highest_pos:
                result.append(this_number - 1)  # 插入当前数字，将原先增加的1减去
                this_number = 0
            num //= div
        result.reverse()
        return result
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        result = 0
        all_len = 0  # 记录字节的长度
        for docId in postings_list:
            length, binary = CompressedPostings.do_number_encoding(docId)
            result = (result << (length * 8) ) | binary  # 将之前的结果和现在的结果拼接
            all_len += length
        return result.to_bytes(all_len, byteorder='big')
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        # 从字节流转换为整数
        num = int.from_bytes(encoded_postings_list, byteorder='big')
        decoded_postings_list = CompressedPostings.do_number_decoding(num)
        return decoded_postings_list
        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [31]:
### Begin your code
encoding_test = CompressedPostings()
encoding_result = encoding_test.encode([1, 2, 3])
print(encoding_result)
print(encoding_test.decode(encoding_result))
### End your code

b'\x02\x03\x04'
[1, 2, 3]


我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [32]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(len(e))
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [33]:
### Begin your code
test_encode_decode([0, 1, 2, 3])
test_encode_decode([1, 2, 90])
test_encode_decode([23, 45, 90, 101, 1000000])
### End your code

4
[0, 1, 2, 3] b'\x01\x02\x03\x04'
3
[1, 2, 90] b'\x02\x03['
7
[23, 45, 90, 101, 1000000] b'\x18.[fA\x84\xbd'


现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [356]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [357]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

第0块时间为:  29.527858018875122
第1块时间为:  27.996601343154907
第2块时间为:  36.111323833465576
第3块时间为:  28.676368474960327
第4块时间为:  30.03512406349182
第5块时间为:  29.272217273712158
第6块时间为:  29.025397300720215
第7块时间为:  23.958435535430908
第8块时间为:  23.351813316345215
第9块时间为:  22.72871994972229
索引构建时间： 281.1043336391449
索引合并时间： 1507.9019174575806


In [45]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

像之前一样，用已构造好的查询语句来测试是否正常运行

In [46]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [34]:
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    # 将单个整数进行编码的函数
    @staticmethod
    def do_number_encoding(num):
        num += 2  # 为了处理0，所有数都+2
        length = math.floor(math.log2(num)) + 1  # 位数
        later = 2 ** (length - 1) - 1  # 一元编码
        former = num & later  # 原二进制除去最高位后的内容
        after_encoding = (former << length ) | later  # 为方便解码，将原二进制的剩余内容放在高位，将一元编码放在低位
        return length * 2 - 1, after_encoding
    
    @staticmethod
    def do_number_decoding(num):
        decoded_postings_list = []
        bit = 8
        bit_num = 256
        # 当解码未结束时
        while num > 0:
            this_length = 0  # 当前整数转换为二进制后的长度-1
            reminder = num % bit_num  # 为加快解码速度，1次取8位
            # 当前位依然是一元编码的内容
            while 1:
                if reminder == big_num - 1:
                    this_length += bit
                    num = num >> bit
                    reminder = num % bit_num
                    continue
                # 当前4位中有0出现
                while 1:
                    # 每两位找0
                    if reminder % 4 == 3:
                        reminder = reminder >> 2
                        num = num >> 2
                        this_length += 2
                        continue
                    # 1位1位找1
                    while reminder % 2 == 1:
                        reminder = reminder >> 1
                        num = num >> 1
                        this_length += 1
                    num = num >> 1
                    break
                break
            biggest_number = 2 ** this_length
            this_num = (num % (biggest_number)) + biggest_number  # 当前正在解码的整数
            decoded_postings_list.append(this_num - 2)
            num = num >> this_length
        return decoded_postings_list
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        result = 0  # 结果
        all_len = 0  # 编码后的总长度
        for docId in postings_list:
            length, binary = ECCompressedPostings.do_number_encoding(docId)  # 获得当前数字编码后的长度和内容
            all_len += length
            result = (result << length) | binary  # 加入结果编码
        return result.to_bytes(math.ceil(all_len / 8), byteorder='big')
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        num = int.from_bytes(encoded_postings_list, byteorder='big')
        decoded_postings_list = ECCompressedPostings.do_number_decoding(num)
        decoded_postings_list.reverse()
        return decoded_postings_list
        ### End your code

测试样例。

In [35]:
### Begin your code
encoding_test = ECCompressedPostings()
encoding_result = encoding_test.encode([0, 1, 2, 3])
print(encoding_result)
print(encoding_test.do_number_decoding(int.from_bytes(encoding_result, byteorder='big')))
print(encoding_test.decode(encoding_result))
### End your code

b'4k'
[3, 2, 1, 0]
[0, 1, 2, 3]


In [36]:
def test_encode_decode_gamma(l):
    e = ECCompressedPostings.encode(l)
    d = ECCompressedPostings.decode(e)
    assert d == l
    print(len(e))  # 输出编码后的长度
    print(l, e)

测试样例。

In [37]:
### Begin your code
test_encode_decode_gamma([0, 1, 2, 3])
test_encode_decode_gamma([1, 2, 90])
test_encode_decode_gamma([23, 45, 90, 101, 1000000])
### End your code

2
[0, 1, 2, 3] b'4k'
3
[1, 2, 90] b'\x14n?'
11
[23, 45, 90, 101, 1000000] b"\x12\xf7\xbe\xe3\xf9\xdf\xf4$'\xff\xff"


创建新目录。

In [389]:
try: 
    os.mkdir('output_dir_ECcompressed')
except FileExistsError:
    pass

In [112]:
BSBI_instance_ECcompressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_ECcompressed', postings_encoding=ECCompressedPostings)
BSBI_instance_ECcompressed.index()

第0块时间为:  31.68158531188965
第1块时间为:  29.765013694763184
第2块时间为:  32.603771686553955
第3块时间为:  27.59824013710022
第4块时间为:  29.69919466972351
第5块时间为:  28.191696166992188
第6块时间为:  26.90467119216919
第7块时间为:  21.81041646003723
第8块时间为:  22.604939222335815
第9块时间为:  21.553518533706665
索引构建时间： 272.8205726146698
索引合并时间： 1404.7573945522308


In [126]:
BSBI_instance_ECcompressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_ECcompressed', postings_encoding=ECCompressedPostings)
BSBI_instance_ECcompressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

In [127]:
for i in range(1, 9):
    start_time = time.time()
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_ECcompressed.retrieve(query)]
        print(time.time() - start_time)  # 输出查询的运行时间
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

45.33920383453369
Results match for query: we are
108.35786318778992
Results match for query: stanford class
122.47082448005676
Results match for query: stanford students
0.6513309478759766
Results match for query: very cool
143.02038073539734
Results match for query: the
94.63334465026855
Results match for query: a
145.02497911453247
Results match for query: the the
114.05165648460388
Results match for query: stanford computer science


# 作业提交

本次作业用时两周，截止日期为11.4。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到×××，命名方式为`学号_姓名_hw4`。